# Quick start: SATRAP CTI analysis toolbox

Import the SATRAP CTI analysis toolbox.

In [ ]:
from satrap.service.satrap_analysis import CTIanalysisToolbox
from satrap.settings import TYPEDB_SERVER_ADDRESS, DB_NAME

satrap = CTIanalysisToolbox(TYPEDB_SERVER_ADDRESS, DB_NAME)


## Starting with simple functions

Get the number of STIX Domain Objects (SDOs) in the knowledge base per STIX type.

In [ ]:
print(satrap.get_sdo_stats())

Print the catalog of MITRE ATT&CK techniques and subtechniques. Subtechniques can be optionally filtered out.

In [ ]:
# print(satrap.mitre_attack_techniques(subtechniques=False))
print(satrap.mitre_attack_techniques())


Print the MITRE ATT&CK catalog of mitigations.

In [ ]:
print(satrap.mitre_attack_mitigations())

Get information on a specific MITRE ATT&CK element (technique, group, software, etc.) using its MITRE ATT&CK id.

In [ ]:
try:
	t1 = satrap.get_attck_concept_info("T1027.001")
	for data in t1:
		print(CTIanalysisToolbox.tabulate_stix_obj(data))
except ValueError as exc:
	print(exc)

Get information about a STIX object using its STIX id.

In [ ]:
try:
	sdo = satrap.search_stix_object("campaign--0c259854-4044-4f6c-ac49-118d484b3e3b")
	print(CTIanalysisToolbox.tabulate_stix_obj(sdo))
except ValueError as e:
	print(e)

Retrieve mitigations explicitly associated to a specific technique using its STIX id.

In [ ]:
mitigations = satrap.mitigations_for_technique("attack-pattern--d1fcf083-a721-4223-aedf-bf8960798d62")
display(mitigations)

## CTI analysis through automated reasoning

The following functions use the inference engine to draw relations on the data explicitly stored in the knowledge base. For some of them, the use of inference is optional.

Get statistics on the usage of ATT&CK techniques by groups. The output of this function is the same as when running the command `satrap techniques` on the CLI.

In [ ]:
# Run without the use of the inference engine
try:
	print("( MITRE ATT&CK technique, Name, Used by (num. intrusion sets) )")
	display(satrap.techniques_usage())
except Exception as err:
	print(err)

In [ ]:
# Run using the inference engine
try:
	print("( MITRE ATT&CK technique, Name, Used by (num. intrusion sets) )")
	display(satrap.techniques_usage(infer=True))
except Exception as err:
	print(err)

Get the techniques used by a specific group. Optionally enable inference.

In [ ]:
techniques = satrap.techniques_used_by_group("G0025", infer=True)
display(techniques)

Obtain the set of courses of action found to mitigate any of the techniques used by a specific group.

In [ ]:
display(satrap.related_mitigations(group_name="BlackTech"))

### Explanation of inferred knowledge

**Hint:** The outcome of an explanation includes the underlying query that was executed by the function. For an interactive visual explanation, we suggest to copy and run the query in TypeDB Studio as explained in the user manual (at `docs/manual/interfaces.md`).

Obtain an explanation for why (and if) a given mitigation is found to address the techniques used by a group. This function can be used to explain individual courses of action in the output of `related_mitigations` shown above.

For instance, to know why "Remote Data Storage" is found to be a course of action against "BlackTech":

In [ ]:
try:
	rel_explanation = satrap.explain_if_related_mitigation("G0098", "course-of-action--20a2baeb-98c2-4901-bad7-dc62d0a03dea")
	print(f"EXECUTED QUERY:\n{rel_explanation.query}\n")
	print("EXPLANATIONS:")
	for tech in rel_explanation.explanations:
		print(CTIanalysisToolbox.format_dict(tech.as_json()), end="\n")
except ValueError as err:
	print(err)

Obtain relevant courses of action for a given intrusion set, when considering the mitigation of a specific SDO (usually technique).

E.g., What mitigations can we find against "ZIRCONIUM" that specifically address the technique "T1059.006"?

In [ ]:
try:
	reason = satrap.explain_related_techniques("ZIRCONIUM", "T1059.006")
	print(f"EXECUTED QUERY\n{reason.query}\n")
	for tech in reason.explanations:
		print(CTIanalysisToolbox.format_dict(tech.as_json()), end="\n")
except Exception as exc:
	print(exc)

Find out, with an explanation, the set of techniques used by a specific group. Optionally filter to get the explanation of a specific technique.

In [ ]:
try:
	dg_explanation = satrap.explain_techniques_used_by_group("G0071", "Domain Groups")
	for tech in dg_explanation.explanations:
		print(CTIanalysisToolbox.format_dict(tech.as_json()), end="\n")
except Exception as err:
	print(err)